In [1]:
import src.TX as TX
import numpy as np
import cv2
import time

#### parameters that needs to be exhanged between the sender and the receiver #####
# IP = "10.42.0.15"
IP = "0.0.0.0"
PORT = 23422
            #  t1  t2  t3  t4  t5  t6  t7  t8  t9
X = np.array([[ 1,  0,  0,  0,  0,  0,  1,  0,  0], # m1
              [ 1,  0,  0,  0,  0,  0,  0,  1,  0], # m2
              [ 1,  0,  0,  0,  0,  0,  0,  0,  1], # m3
              [ 0,  1,  0,  0,  0,  0,  1,  0,  0], # m4
              [ 0,  1,  0,  0,  0,  0,  0,  1,  0], # m5
              [ 0,  1,  0,  0,  0,  0,  0,  0,  1], # m6
              [ 0,  0,  1,  0,  0,  0,  1,  0,  0], # m7
              [ 0,  0,  1,  0,  0,  0,  0,  1,  0], # m8
              [ 0,  0,  1,  0,  0,  0,  0,  0,  1], # m9
              [ 0,  0,  0,  1,  0,  0,  1,  0,  0], # m10
              [ 0,  0,  0,  1,  0,  0,  0,  1,  0], # m11
              [ 0,  0,  0,  1,  0,  0,  0,  0,  1], # m12
              [ 0,  0,  0,  0,  1,  0,  1,  0,  0], # m13
              [ 0,  0,  0,  0,  1,  0,  0,  1,  0], # m14
              [ 0,  0,  0,  0,  1,  0,  0,  0,  1], # m15
              [ 0,  0,  0,  0,  0,  1,  1,  0,  0], # m16
              [ 0,  0,  0,  0,  0,  1,  0,  1,  0], # m17
              [ 0,  0,  0,  0,  0,  1,  0,  0,  1]]) # m18
            #  t1  t2  t3  t4  t5  t6  t7  t8  t9
Y = np.array([[ 0,  0,  0,  0,  0,  0,  1,  0,  0], # m1
              [ 0,  0,  0,  0,  0,  0,  0,  1,  0], # m2
              [ 1,  0,  0,  0,  0,  0,  0,  0,  0], # m3
              [ 0,  0,  0,  0,  0,  0,  0,  0,  0], # m4
              [ 0,  0,  0,  0,  0,  0,  0,  0,  0], # m5
              [ 0,  1,  0,  0,  0,  0,  0,  0,  0], # m6
              [ 0,  0,  0,  0,  0,  0,  0,  0,  0], # m7
              [ 0,  0,  0,  0,  0,  0,  0,  0,  0], # m8
              [ 0,  0,  1,  0,  0,  0,  0,  0,  0], # m9
              [ 0,  0,  0,  0,  0,  0,  0,  0,  0], # m10
              [ 0,  0,  0,  0,  0,  0,  0,  0,  0], # m11
              [ 0,  0,  0,  1,  0,  0,  0,  0,  0], # m12
              [ 0,  0,  0,  0,  0,  0,  0,  0,  0], # m13
              [ 0,  0,  0,  0,  0,  0,  0,  0,  0], # m14
              [ 0,  0,  0,  0,  1,  0,  0,  0,  0], # m15
              [ 0,  0,  0,  0,  0,  0,  0,  0,  0], # m16
              [ 0,  0,  0,  0,  0,  1,  0,  0,  0], # m17
              [ 0,  0,  0,  0,  0,  0,  0,  0,  1]]) # m18


# X = np.eye(18)
# Y = np.eye(18)
chunk_size_Byte = 1024
key = b"key"
digestmod = 'sha384'


udp_tx = TX.UDP_TX(IP=IP, PORT=PORT, X=X, Y=Y, chunk_size_Byte=chunk_size_Byte, KEY=key, digestmod=digestmod)

# with open("send.txt", "rb") as f:
#     udp_tx.transmit(f.read())

# udp_tx.transmit(b"Hello World this is not working and this is bad, but I think I need a bigger text to transmit", attack=[0])
# 1/0

cap = cv2.VideoCapture(0)  # Capture video from the default camera
# Define the compression parameters for Progressive JPEG
compression_params = [cv2.IMWRITE_JPEG_PROGRESSIVE, 1, cv2.IMWRITE_JPEG_QUALITY, 70]
while True:
    ret, frame = cap.read()
    # Encode the image to JPEG format in memory
    success, encoded_frame = cv2.imencode('.jpg', frame, compression_params)
    data = encoded_frame.tobytes()
    print(len(data))
    udp_tx.transmit(data)
    cv2.imshow('Webcam', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to quit
        break
    time.sleep(.01)
cap.release()



[ WARN:0@0.365] global ./modules/videoio/src/cap_gstreamer.cpp (1405) open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1


39855
40125
40156
40397
40444
40480
40530
40460
40521
40468
40505
40475
40440
40406
40455
40400
40381
40411
40395
40420
40457
40426
40374
40373
40380
40317
40281
40357
40375
40354
40314
40319
40336
40320
40339
40364
40267
40340
40310
40304
40313
40273
40307
40282
40263
40289
40268
40173
40189
40118
40073
40042
40054
39991
39995
40145
39970
40009
39892
39907
39978
39966
40071
40037
39968
39350
33638
30734
28022
27990
28697
34030
36470
38084
39113
39517
39707
39767
39831
39777
39718
39752
39711
39683
39603
39581
39519
39558
39576
39579
39641
39654
39710
39685
39583
39564
39537
39523
39518
39563
39534
39541
39657
39689
39704
39808
39876
39931
39930
39949
39884
39959
40037
40062
40077
40117
40120
40197
40171
40163
40279
40239
40313
40385
40377
40323
40327
40402
40386
40337
40325
40297
40258
40214
40209
40208
40142
40088
40121
40046
40027
40059
40126
40093
39999
40106
40089
40089
40085
40229
40190
40259
40297
40305
40346
40391
40330
40388
40351
40477
40371
40410
40435
40353
40434
40397
4041